# Marketing Mix Modeling

In this notebook, we will look at Marketing Mix Models as an example of the usage of linear regression in Business Intelligence.

The task: *We want to model sales (or another KPI such as number of customers, leads, etc.) as a function of advertising, trends, events, seasonality, price, distribution, macro-economic factors, the weather etc.*

Notes:
* For offline medias (traditional medias such TV, Radio, outdoor banners etc.), it can be really hard to measure the effect of marketing on sales, because we never know when people buy a product, whether they do it (partly) because they saw a TV ad. (We can measure some of it with special discount codes, however.)
* The different variables comes with different controls from our side
    + Seasonality, macro-economic factors and the weather, we cannot control
    + Price and distribution can be controlled to some extend
    + Advertising is where we have the most control
* With such a model, we can evaluate marketing campaigns and plan future campaign based on which marketing mix that we predicted to generate most sales
* A classical approach is to use some sort of Linear Regression.
* See the blogs: https://analyticsartist.wordpress.com/2014/08/17/marketing-mix-modeling-explained-with-r/ and https://analyticsartist.wordpress.com/2013/11/02/calculating-adstock-effect/

In [ ]:
library(tidyverse)
options(repr.plot.width=8, repr.plot.height=5)

## A simple first model

The simplest model we can think off is:

$$
Sales = Base + b_1*Advertising ,
$$

where $Base$ represents the base sales we experience when there is no advertisement.

In [ ]:
sales <- c(37, 89, 82, 58, 110, 77, 103, 78, 95, 106, 98, 96, 68, 96, 157, 198, 145, 132, 96, 135)
ad <- c(6, 27, 0, 0, 20, 0, 20, 0, 0, 18, 9, 0, 0, 0, 13, 25, 0, 15, 0, 0)
modFit0 <- lm(sales~ad)
summary(modFit0)
ggplot(mapping = aes(x = ad, y = sales)) + geom_point() + geom_smooth(method = "lm")

You might wonder what the different values of the `ad` variable of the previous model refer to. In principle, it could be many different things, however, it needs to be something we somewhat can control and that is a measure for the exposure of the ad. What matters for sales is how many times the add was shown and how many saw it. This can be measured/estimated in several different ways such as in [TRPs](https://marketing-dictionary.org/t/target-rating-point/), [GRPs](https://marketing-dictionary.org/g/gross-rating-point/), [Impressions](https://marketing-dictionary.org/a/advertising-impression/), or simply just the media spend (the amount paid for the ads).

## Adding adstock transformation

The effect of an ad is not limited to the week (or the day) the ad is shown.
- The effect *carries over* to the following weeks (days) with a decay.
- Explanation of this effect: Exposures to an ad build awareness in consumers' minds. This awareness doesn't disappear right away, but remains in the consumers' memory. However, memory decays over time.
- The adstock formula is:

$$
AdEffect(t) = AdSpend(t) + CarryOver * AdEffect(t-1)
$$

In [ ]:
ad_adstock <- as.numeric(stats::filter(x=ad, filter=.50, method="recursive"))
ggplot(mapping = aes(x = seq(1,length(ad)), y = ad)) +
  geom_col() + ylab("Advertising") + xlab("Time (in Weeks)") +
  geom_line(mapping = aes(x = seq(1,length(ad)), y = ad_adstock), size = 2, color = "blue")

A few comments.
- First of all, to create the adstock we use the `filter` function from base R. However, as we already loaded the `tidyverse` package this filter function has been overwritten by the `filter` function we have used in the previous lectures. Thus to use the filter function from base R we use `stats::filter` as filter in base R is located in a package called `stats`. This syntax can also be used for other packages, so we could just write `dplyr::filter` when we want to use the filter function we have previously used instead of loading the package `tidyverse` (which the package `dplyr` is part of).
- In interpreting the above plot, the bars represent weekly spend (for instance) on advertisement, while the blue curve represent the ad-effect or the "awareness of the ad" (i.e. the adstock effect).
- In the above plot the carryover parameter is set to $0.5$ (try to locate that value in the code), which means that 50% of the ad-effect is carried over to the next week.

For different values of the `CarryOver` parameter we get fairly different awareness effect:

In [ ]:
ad_adstock <- as.numeric(stats::filter(x=ad, filter=.25, method="recursive"))
ggplot(mapping = aes(x = seq(1,length(ad)), y = ad)) +
  geom_col() + ylab("Advertising") + xlab("Time (in Weeks)") +
  geom_line(mapping = aes(x = seq(1,length(ad)), y = ad_adstock), size = 2, color = "blue") +
  ggtitle("Carry-over of 0.25")

ad_adstock <- as.numeric(stats::filter(x=ad, filter=.75, method="recursive"))
ggplot(mapping = aes(x = seq(1,length(ad)), y = ad)) +
  geom_col() + ylab("Advertising") + xlab("Time (in Weeks)") +
  geom_line(mapping = aes(x = seq(1,length(ad)), y = ad_adstock), size = 2, color = "blue") +
  ggtitle("Carry-over of 0.75")

## Adding adstock transformation

We can now add the adstock transformation of the advertising to the model. With the ad stock transformation, the model now looks like this:

$$
Sales = Base + b1*Adstock(Advertising, a),
$$

where $a$ is the adstock parameter that determines how strong the carry-over should be.

If we for now set the adstock parameter $a$ to $0.5$ we can build the following model:

In [ ]:
ad_adstock <- as.numeric(stats::filter(x=ad, filter=.50, method="recursive"))
modFit1 <- lm(sales ~ ad_adstock)
summary(modFit1)
ggplot(mapping = aes(x = ad_adstock, y = sales)) + geom_point() + geom_smooth(method = "lm")

## Adding another media

We can have multiple medias in the model. For instance, the model could instead look like this:

$$
Sales = Base + b_1*Advertising_1 + b_2*Advertising_2 .
$$

Note, the different medias/advertisements could have different carry overs.

If we try with an adstock carry-over parameter of $0.3$ for both medias, we get the following model:

In [ ]:
ad1 <- ad
ad2 <- c(3, 0, 4, 0, 5, 0, 0, 0, 8, 0, 0, 5, 0, 11, 16, 11, 5, 0, 0, 15)
ad1_adstock <- as.numeric(stats::filter(x=ad1, filter=.3, method="recursive"))
ad2_adstock <- as.numeric(stats::filter(x=ad2, filter=.3, method="recursive"))
modFit2 <- lm(sales~ad1_adstock+ad2_adstock)
summary(modFit2)

## Adding trend

There might be a trend in the response variable sales (it can be due to several things). Let us a have look at sales over time:

In [ ]:
ggplot(mapping = aes(x = 1:length(sales), y = sales)) + geom_line()

It is a bit hard to see from this plot whether there is a trend. It could look a bit like an upward trend though. 

Let us have a look at the residual plots for the two models instead:

In [ ]:
ggplot(mapping = aes(x = 1:length(modFit1$residuals), y = modFit1$residuals)) + geom_point()
ggplot(mapping = aes(x = 1:length(modFit2$residuals), y = modFit2$residuals)) + geom_point()

Here the trend seems much clearer and it would probably make sense to add a trend to the model. So let us add trend to the model:

In [ ]:
trend <- 1:length(sales)
modFit3 <- lm(sales~trend+ad1_adstock+ad2_adstock)
summary(modFit3)

In [ ]:
1:length(sales)

Note how we have added the trend to the model!

Let us look at the residuals again:

In [ ]:
ggplot(mapping = aes(x = 1:length(modFit3$residuals), y = modFit3$residuals)) + geom_point()
ggplot() + geom_histogram(aes(x = modFit3$residuals), binwidth = 4)

The first plot shows the residuals over time and show no clear patterns. Thus, there is probably no seasonality or trend in the sales data that we have not captured in our model. The second plot shows that our residuals are also almost normally distributed, which is what we want (recall the model assumptions for linear regression.)

## Interpreting the results

Let us now start interpreting the results.

First we will simply sum the effect of each of the variables ("base", "ad1_adstock", "ad2_adstock", "trend") to get an idea about how much the sales was driven by each one of the in the period in total.

In [ ]:
total_effect <- data.frame(Variable = c("base", "ad1_adstock", "ad2_adstock", "trend"),
                           Effect = c(modFit3$coefficients["(Intercept)"] * length(sales),
                                      sum(modFit3$coefficients["ad1_adstock"] * ad1_adstock),
                                      sum(modFit3$coefficients["ad2_adstock"] * ad2_adstock),
                                      sum(modFit3$coefficients["trend"] * trend)))
ggplot(total_effect, aes(Variable, Effect)) + geom_col() + ggtitle("Total effect")

From this plot we can see that most of the sales is driven by a base sales and a trend. However, the two ad campaigns did also drive a significant amount of sales. Ad1 seems to have driven more sales than ad2.

We can also sum the two original variables `ad1` and `ad2` as they represented that actual spend on the two medias. Thus we will for each ad get the total spend in the period.

In [ ]:
total_spend <- data.frame(Media = c("ad1", "ad2"),
                          Spend = c(sum(ad1), sum(ad2)))
ggplot(total_spend, aes(Media, Spend)) + geom_col() + ggtitle("Total Media Spend")

So while, ad1 drove more sales than ad2, we also spend more money on ad1 than on ad2. Thus we should find another more precise way of comparing the two ads.

One such way of comparing could be on "Cost per sales" (CPS), that is for each of the two ads, what does it cost us to generate one extra sale?

In [ ]:
cps_df <- data.frame(Media = c("ad1", "ad2"),
                     CPS = c(total_spend$Spend[1]/sum(total_effect$Effect[2]),
                             total_spend$Spend[2]/sum(total_effect$Effect[3])))

ggplot(cps_df, aes(Media, CPS)) + geom_col() + ggtitle("Cost per Sales (CPS)")

Here we see that it cost us more to generate an extra sale with ad1 than with ad2. In this sense ad2 seems more effective.

Another way of evaluating the two medias would be on "return on investment" (ROI). So let us do this.

The formula for ROI is:

`ROI = (gain from investment - cost of investment) / cost of investment`

To use this, we need to know what the "gain from investment" is. This is the profit margin (What the company makes on each sold product). For now, we will assume the profit margin is `0.5` EUROs. Then we can calculate the total ROIs of each ad:

In [ ]:
roi_df <- data.frame(Media = c("ad1", "ad2"),
                          ROI = c((0.5 * sum(modFit3$coefficients["ad1_adstock"] * ad1_adstock) -
                                     total_spend$Spend[1])/total_spend$Spend[1],
                                  (0.5 * sum(modFit3$coefficients["ad2_adstock"] * ad2_adstock) -
                                     total_spend$Spend[2])/total_spend$Spend[2]))
ggplot(roi_df, aes(Media, ROI)) + geom_col() + ggtitle("Return on Investment (ROI)")

From this we can see that for every extra dollar we invest in ad1, we will make profit of around 0.5 cents, while for every dollar extra we invest in ad2, we will make a profit of around 1 dollar. Thus, ad2 is much more efficient in this sense.

A note is in order here. It seems like we could just invest all the money we can get in ad2 and thereby increase our profit. However, there is also a saturation effect to advertisement we have not included in the model. At some point the market is saturated and we cannot sell more products or there are simply no more room in the media for our ads (all tv spots are full of it). Thus, under large increase in advertisement spend, we are likely to see a drop in ROIs. For more, see for instance [this blog post](https://analyticsartist.wordpress.com/2015/03/08/advertising-diminishing-returns-saturation/).

### *Exercise*

In our final model `sales~trend+ad1_adstock+ad2_adstock` we used an carryover of `0,3` for both `ad1_adstock` and `ad2_adstock` and recieved an adjusted R-square of `0.9292`. Go back and try out different values for the carry-over of the two medias and see iff you can find a better model. (If you want to be systematic, you can try and do some nested for loops.)